In [19]:
!pip install googlemaps
!pip install polyline
!pip install folium

import pandas as pd
import math
import scipy.optimize as opt
from geopy.geocoders import GoogleV3
import googlemaps
import polyline
import folium

api_key = ''

# Initialize the Google Maps geolocator and client
geolocator = GoogleV3(api_key=api_key)
gmaps = googlemaps.Client(key=api_key)

In [3]:
from geopy.geocoders import Nominatim
import time

geolocator = Nominatim(user_agent="atlanta_locator")

latitudes = []
longitudes = []

for neighborhood in df['Neighborhood']:
    location = geolocator.geocode(f"{neighborhood}, Atlanta, GA")
    if location:
        latitudes.append(location.latitude)
        longitudes.append(location.longitude)
    else:
        latitudes.append(None)
        longitudes.append(None)
    time.sleep(1) 

df['Latitude'] = latitudes
df['Longitude'] = longitudes

print(df)


             Neighborhood  Population   Latitude  Longitude
0                 Midtown       16569  33.781656 -84.384071
1                Downtown       13411  33.763819 -84.385607
2         Old Fourth Ward       10505  33.764108 -84.371763
3          North Buckhead        8270  33.877277 -84.366380
4              Pine Hills        8033  33.563652 -84.259534
5  Morningside/Lenox Park        8030  33.805429 -84.356930
6       Virginia-Highland        7800  33.782656 -84.353691
7              Grant Park        6771  33.735862 -84.370932
8            Georgia Tech        6607  33.776095 -84.398808
9                Kirkwood        5897  33.752427 -84.323154


In [21]:
# Create DataFrame with neighborhood names
data = {
    'Location': [
        'Midtown, Atlanta, GA', 
        'Downtown, Atlanta, GA', 
        'Old Fourth Ward, Atlanta, GA',
        'North Buckhead, Atlanta, GA', 
        'Pine Hills, Atlanta, GA', 
        'Morningside/Lenox Park, Atlanta, GA', 
        'Virginia-Highland, Atlanta, GA', 
        'Grant Park, Atlanta, GA', 
        'Georgia Tech, Atlanta, GA', 
        'Kirkwood, Atlanta, GA'
    ]
}

df = pd.DataFrame(data)
# Function to get geocode for a location
def get_geocode(location):
    coords = geolocator.geocode(location)
    lat = round(coords.latitude, 4)
    lng = round(coords.longitude, 4)
    return lat, lng

# Get geocodes for each neighborhood
df[['Lat', 'Lng']] = df['Location'].apply(lambda x: pd.Series(get_geocode(x)))

# Display the DataFrame with coordinates
print(df)


                              Location      Lat      Lng
0                 Midtown, Atlanta, GA  33.7833 -84.3831
1                Downtown, Atlanta, GA  33.7557 -84.3884
2         Old Fourth Ward, Atlanta, GA  33.7640 -84.3720
3          North Buckhead, Atlanta, GA  33.8527 -84.3654
4              Pine Hills, Atlanta, GA  33.8375 -84.3516
5  Morningside/Lenox Park, Atlanta, GA  33.7962 -84.3595
6       Virginia-Highland, Atlanta, GA  33.7817 -84.3635
7              Grant Park, Atlanta, GA  33.7372 -84.3682
8            Georgia Tech, Atlanta, GA  33.7756 -84.3963
9                Kirkwood, Atlanta, GA  33.7533 -84.3262


In [23]:
# Great circle (Haversine) distance function
def calc_dist_haversine(lat1, lng1, lat2, lng2):
    lat1, lng1, lat2, lng2 = map(math.radians, [lat1, lng1, lat2, lng2])
    a = math.sin((lat2 - lat1) / 2) ** 2 + math.cos(lat1) * math.cos(lat2) * math.sin((lng2 - lng1) / 2) ** 2
    dist_haversine_miles = 3959 * 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    return dist_haversine_miles
# Function to calculate total Haversine distance from a point to all neighborhoods
def calc_cost_haversine(coords, data):
    lat, lon = coords
    distances = []
    for _, row in data.iterrows():
        distances.append(calc_dist_haversine(lat, lon, row['Lat'], row['Lng']))
    return sum(distances)

# Initial guess (center of all neighborhoods)
initial_guess = [df['Lat'].mean(), df['Lng'].mean()]

# Optimize to find the minimum total Haversine distance
result = opt.minimize(calc_cost_haversine, initial_guess, args=(df,), method='SLSQP')

# Get the optimal location address
result_address = gmaps.reverse_geocode((result.x[0], result.x[1]))[0]['formatted_address']

print(f"Haversine Optimal Pickup Location: {result_address} at coordinates {result.x.round(4)}")


Haversine Optimal Pickup Location: 515 8th St NE, Atlanta, GA 30308, USA at coordinates [ 33.7793 -84.3702]


In [25]:
# Function to calculate driving distance using Google Maps Directions API
def calc_dist_driving(lat1, lng1, lat2, lng2):
    directions = gmaps.directions((lat1, lng1), (lat2, lng2), mode='driving')
    dist_miles = directions[0]['legs'][0]['distance']['value'] / 1609.344  # Convert meters to miles
    return dist_miles
# Function to calculate total driving distance from a point to all neighborhoods
def calc_cost_driving(coords, data):
    lat, lon = coords
    distances = []
    for _, row in data.iterrows():
        distances.append(calc_dist_driving(lat, lon, row['Lat'], row['Lng']))
    return sum(distances)

# Optimize to find the minimum total driving distance
result_driving = opt.minimize(calc_cost_driving, initial_guess, args=(df,), method='SLSQP')

# Get the optimal driving location address
result_driving_address = gmaps.reverse_geocode((result_driving.x[0], result_driving.x[1]))[0]['formatted_address']

print(f"Driving Optimal Pickup Location: {result_driving_address} at coordinates {result_driving.x.round(4)}")


Driving Optimal Pickup Location: 600 Cresthill Ave NE, Atlanta, GA 30306, USA at coordinates [ 33.7837 -84.3674]


In [33]:
# Initialize the map centered around the mean of all locations
m = folium.Map(location=initial_guess, zoom_start=12)
# Add markers for optimal locations
folium.Marker(result.x, popup=result_address, icon=folium.Icon(color='orange')).add_to(m)  # Haversine optimal point
folium.Marker(result_driving.x, popup=result_driving_address, icon=folium.Icon(color='green')).add_to(m)  # Driving optimal point
# Add markers for all neighborhoods
for i in range(len(df)):
    folium.Marker([df['Lat'][i], df['Lng'][i]], popup=df['Location'][i]).add_to(m)
# Draw driving paths from the driving optimal point to all neighborhoods
for i in range(len(df)):
    start = tuple(result_driving.x)
    end = (df['Lat'][i], df['Lng'][i])

    # Get directions from Google Maps API
    directions = gmaps.directions(start, end, mode="driving")

    # Decode the polyline points from the API response
    points = polyline.decode(directions[0]['overview_polyline']['points'])

    # Add the driving path to the map
    folium.PolyLine(locations=points, color='blue', weight=5).add_to(m)
# Save the map as HTML
m.save('C:\\Users\\17069\\Desktop\\Supply Chain\\atlanta_pickup_locations.html')
print(f"\nCost using Haversine distance from optimal point: $ {result.fun:.2f}; Coordinates: {result.x.round(4)}; Address: {result_address}")
print(f"\nCost using Driving distance from optimal point: $ {result_driving.fun:.2f}; Coordinates: {result_driving.x.round(4)}; Address: {result_driving_address}")



Cost using Haversine distance from optimal point: $ 22.30; Coordinates: [ 33.7793 -84.3702]; Address: 515 8th St NE, Atlanta, GA 30308, USA

Cost using Driving distance from optimal point: $ 28.09; Coordinates: [ 33.7837 -84.3674]; Address: 600 Cresthill Ave NE, Atlanta, GA 30306, USA
